In [26]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.linear_model import LassoCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [27]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [28]:
train.set_index('Claim Identifier', inplace=True)
train.drop_duplicates(inplace=True)
train.drop('OIICS Nature of Injury Description', axis=1, inplace=True)

#train.drop(columns=['Birth Year', 'Age at Injury', 'Number of Dependents', 'WCIO Cause of Injury Code'], inplace=True)

In [29]:
train_num = train.select_dtypes(include=np.number).columns.tolist()
train_cat = train.select_dtypes(exclude=np.number).columns.tolist()

In [30]:
# Numerical columns: Impute with mean
num_imputer = SimpleImputer(strategy="mean")
train[train_num] = pd.DataFrame(
    num_imputer.fit_transform(train[train_num]),
    columns=train_num,
    index=train.index
)

# Categorical columns: Impute with most frequent (mode)
cat_imputer = SimpleImputer(strategy="most_frequent")
train[train_cat] = pd.DataFrame(
    cat_imputer.fit_transform(train[train_cat]),
    columns=train_cat,
    index=train.index
)

In [31]:
X = train.drop(columns='Claim Injury Type')
y = train['Claim Injury Type']

In [32]:
X_num = X.select_dtypes(include=np.number).set_index(X.index)
X_cat = X.select_dtypes(exclude=np.number).set_index(X.index)

In [33]:
min_max = MinMaxScaler()
min_max.fit(X_num) #fit to training data
X_num_scaled_min_max = min_max.transform(X_num) # this will return an array
X_num_scaled_min_max = pd.DataFrame(X_num_scaled_min_max, columns = X_num.columns).set_index(X.index) # Convert the array to a pandas dataframe

min_max2 = MinMaxScaler(feature_range=(-1, 1))
min_max2.fit(X_num) #fit to training data
X_num_scaled_min_max2 = min_max2.transform(X_num) # this will return an array
X_num_scaled_min_max2 = pd.DataFrame(X_num_scaled_min_max2, columns = X_num.columns).set_index(X.index) # Convert the array to a pandas dataframe

# StandardScaler
standard_scaler = StandardScaler()
standard_scaler.fit(X_num) # fit to training data
X_num_scaled_standard = standard_scaler.transform(X_num) # this will return an array
X_num_scaled_standard = pd.DataFrame(X_num_scaled_standard, columns=X_num.columns).set_index(X.index) # Convert the array to a pandas dataframe

# RobustScaler
robust_scaler = RobustScaler()
robust_scaler.fit(X_num) # fit to training data
X_num_scaled_robust = robust_scaler.transform(X_num) # this will return an array
X_num_scaled_robust = pd.DataFrame(X_num_scaled_robust, columns=X_num.columns).set_index(X.index) # Convert the array to a pandas dataframe


In [34]:
X_cat = X_cat.astype(str)

enc1 = OrdinalEncoder() #encoder for features
enc2 = LabelEncoder() #encoder for labels
enc1.fit(X_cat)
X_cat_encoded = pd.DataFrame(enc1.transform(X_cat), columns = X_cat.columns).set_index(X.index)
y_encoded = enc2.fit_transform(y)

In [35]:
X_minmax = pd.concat([X_num_scaled_min_max, X_cat_encoded], axis=1)
X_minmax2 = pd.concat([X_num_scaled_min_max2, X_cat_encoded], axis=1)
X_standard = pd.concat([X_num_scaled_standard, X_cat_encoded], axis=1)
X_robust = pd.concat([X_num_scaled_robust, X_cat_encoded], axis=1)
y_encoded_df = pd.DataFrame(y_encoded, columns=['Claim Injury Type'])

In [36]:
def scalers(data_scaled, names):
    for i, name in zip(data_scaled, names):
        print(name)
        X_train, X_val, y_train, y_val = train_test_split(i, y_encoded_df, test_size=0.2, stratify=y_encoded, random_state=5)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        labels_train = model.predict(X_train)
        labels_val = model.predict(X_val)
        print(f1_score(y_train, labels_train, average='macro'))
        print(f1_score(y_val, labels_val, average='macro'))

In [37]:
scalers([X_minmax, X_minmax2, X_standard, X_robust], ['minmax', 'minmax2', 'standard', 'robust'])

minmax
1.0
0.3789991801900998
minmax2
1.0
0.3868077735332589
standard
1.0
0.38678063703399973
robust
1.0
0.3853890591724868
